In [1]:
import pdb
import os
import pickle
import numpy as np
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
from deepnets.convdata import Convdata
from deepnets.convnet import Convnet

model_num   = 3
model_name  = 'alexnet'
hyperparams = { 'batch_size'    : 64,
                'optimizer'     : 'Adam',
                'lr'            : 0.001,
                'lr_stepsize'   : 100,
                'lr_gamma'      : 0.1  }


def get_optim(params_grad):
    if hyperparams['optimizer'].startswith('SGD'):
        optimizer = optim.SGD(params_grad, lr=hyperparams['lr'], momentum=0.9)
    elif hyperparams['optimizer'].startswith('Adam'):
        optimizer = optim.Adam(params_grad, lr=hyperparams['lr'])
    else:
        raise NotImplementedError
    return optimizer

## don't change 
PAGE_SIZE   = 5
NUM_CLASSES = 4
IMG_DIR     = os.sep + os.path.join('projects', 'Medical', 'DigiPath', 'caner', 'ConsROI_patches')
IMG_DIR     = os.path.join(IMG_DIR, 'page'+str(PAGE_SIZE))
MODEL_ID    = f'{model_name}_model{model_num}'
MODEL_DIR   = os.path.join(IMG_DIR, 'models', MODEL_ID)
##


In [2]:
# load datasets
alexdata = Convdata(model_name=model_name, data_path=IMG_DIR)
alexdata(batch_size = hyperparams['batch_size'])


In [3]:
# get pre-trained convnet
alexnet = Convnet(model_name=model_name, pre_trained=True, num_classes=NUM_CLASSES, model_dir=MODEL_DIR)


Running on CPU...


In [76]:
a = 5
a++

SyntaxError: invalid syntax (<ipython-input-76-1329448364ce>, line 2)

In [74]:
dirnet = alexnet.model_dir
fname  = 'finetuned(alexnet)model_epoch(100)'
fpath = os.path.join(dirnet, fname)
aux = fpath
idx = 1
while os.path.isfile(aux):
    aux = fpath + '_v' + str(idx)
    idx += 1
fpath = aux
print(fpath)        

/projects/Medical/DigiPath/caner/ConsROI_patches/page5/models/alexnet_model3/finetuned(alexnet)model_epoch(100)_v1


In [49]:
dirnet = alexnet.model_dir
if dirnet[-3:].startswith('_v'):
    dirnet = dirnet[:-1] + str(int(dirnet[-1])+1)
else:
    dirnet += '_v1'
print(dirnet)

/projects/Medical/DigiPath/caner/ConsROI_patches/page5/models/alexnet_model3_v10_v8


In [33]:
import torch
alexnet.model.load_state_dict(torch.load('/projects/Medical/DigiPath/caner/ConsROI_patches/page5/models/alexnet_model3/finetuned(alexnet)model_epoch(100)'))

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [5]:
alexnet.load_model(epoch=100)
print(list(alexnet.get_params())[-2])

/projects/Medical/DigiPath/caner/ConsROI_patches/page5/models/alexnet_model3/finetuned(alexnet)model_epoch(100)


AttributeError: 'str' object has no attribute 'items'

In [11]:

# loss
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
params_grad = filter(lambda p: p.requires_grad, alexnet.get_model().parameters())
optimizer   = get_optim(params_grad)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=hyperparams['lr_stepsize'], gamma=hyperparams['lr_gamma'])



In [29]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from deepnets.convdata import ConvData
from deepnets.convnet import ConvNet

PAGE_SIZE = 3
NUM_CLASSES = 4
IMG_DIR = os.sep + os.path.join('projects', 'Medical', 'DigiPath', 'caner', 'ConsROI_patches')
IMG_DIR = os.path.join(IMG_DIR, 'page'+str(PAGE_SIZE))

In [1]:
dataObj = ConvData('alexnet', IMG_DIR)
dataObj(batch_size = 64)
dataloaders = dataObj.getData()

# get pre-trained convnet
convnet = ConvNet(convname='alexnet', pre_trained=True, num_classes=4)
model = convnet.get_model()

# loss
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
params = model.parameters()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, params), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
imgs, lbls = dataObj.getImages('train')
lbls = np.array(lbls).reshape((8,8))
plt.rcParams['figure.figsize'] = [15, 15]

r_s, inc = 2, 224
for i in range(8): 
    r_e  = r_s + inc
    img_row = imgs[r_s:r_e,::]
    plt.imshow(img_row)
    plt.title(lbls[i])
    plt.show()
    plt.pause(0.05)
    r_s = r_e + 2
    #r_e = r_e + inc + 1

In [45]:
num_epochs = 25
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            scheduler.step()
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for i, (inputs, labels) in enumerate(dataloaders[phase]):
            inputs, labels = Variable(inputs), Variable(labels)
            #inputs = inputs.to(device)
            #labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            # with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                optimizer.step()

            # statistics
            # print statistics
            running_loss += loss
            if i % 5 == 4:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0


            # running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

NameError: name 'epoch' is not defined

In [ ]:
# loss
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# train
model_best = model_ft.fit(dataloaders, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)